Useful links

1. For the architecture https://towardsdatascience.com/deep-learning-for-specific-information-extraction-from-unstructured-texts-12c5b9dceada
2. https://androidkt.com/multi-label-text-classification-in-tensorflow-keras/
3. https://keras.io/preprocessing/sequence/
4. https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/ ( Not really)
5. For deep learning using word embeddings https://stackabuse.com/python-for-nlp-multi-label-text-classification-with-keras/



In [1]:
import spacy
import pandas as pd
from tqdm import tqdm

In [2]:
DATA_DIR = "../../data/processed/"
INPUT_FILE_NAME = 'final_squash15_with_pos_ner_tm.parquet'

In [3]:
df = pd.read_parquet(DATA_DIR + INPUT_FILE_NAME)
df.head()

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,sim_tags,squash15_tags,pos_sequence,ner_sequence,tm
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"b'[""thank"", ""chris"", ""truly"", ""great"", ""honor""...",thank chris truly great honor opportunity come...,"cars,solar system,energy,culture,politics,scie...","culture,politics,science,global issues,technology",VERB PROPN ADV ADJ NOUN NOUN VERB NOUN ADV ADV...,PERSON ORG ORG GPE LOC ORG PRODUCT GPE GPE PER...,"[0.04325945698517057, 0.0, 0.00142482934694180..."
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"b'[""term"", ""invention"", ""like"", ""tell"", ""tale""...",term invention like tell tale favorite project...,"macarthur grant,simplicity,design,solar system...","design,global issues",NOUN NOUN SCONJ VERB PROPN ADJ NOUN VERB NOUN ...,GPE DATE CARDINAL DATE ORG PERSON LOC ORG GPE ...,"[0.013287880838036227, 0.0, 0.0, 0.00511725094..."
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"b'[""public"", ""dewey"", ""long"", ""ago"", ""observe""...",public dewey long ago observe constitute discu...,"corruption,inequality,science,investment,war,c...","science,culture,politics,global issues,business",ADJ PROPN ADV ADV VERB ADJ NOUN NOUN PROPN PRO...,DATE NORP ORDINAL DATE MONEY DATE DATE DATE EV...,"[0.0, 0.006699599134802422, 0.0, 0.00564851883..."
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"b'[""want"", ""start"", ""say"", ""houston"", ""problem...",want start say houston problem enter second ge...,"flight,design,nasa,science,invention,entrepren...","design,science,business",VERB NOUN VERB PROPN NOUN VERB ADJ NOUN NOUN N...,GPE ORDINAL ORG PERSON DATE DATE DATE TIME PER...,"[0.040282108339079505, 0.03732895646484358, 0...."
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"b'[""want"", ""talk"", ""background"", ""idea"", ""car""...",want talk background idea car art actually mea...,"cars,design,transportation,invention,technolog...","design,technology,business,science",VERB NOUN NOUN NOUN NOUN NOUN ADV ADJ NOUN NOU...,PERSON PRODUCT ORG ORG PERSON PERSON PERSON OR...,"[0.08049208168957463, 0.0, 0.0, 0.008031187136..."


In [4]:
df.iloc[:,:14].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2313 entries, 0 to 2312
Data columns (total 14 columns):
speaker                    2313 non-null object
headline                   2313 non-null object
description                2313 non-null object
duration                   2313 non-null object
tags                       2313 non-null object
transcript                 2313 non-null object
WC                         2313 non-null float64
clean_transcript           2313 non-null object
clean_transcript_string    2313 non-null object
sim_tags                   2313 non-null object
squash15_tags              2313 non-null object
pos_sequence               2313 non-null object
ner_sequence               2313 non-null object
tm                         2313 non-null object
dtypes: float64(1), object(13)
memory usage: 253.1+ KB


In [5]:
def print_full_dataframe(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
    
def compute_tag_ratio(target_column, df=df):
    tags = df[target_column].str.replace(', ',',').str.lower().str.strip()
    split_tags = tags.str.split(',')
    tag_counts_per_talk = split_tags.apply(len)

    joined_tags = tags.str.cat(sep=',').split(',')
    all_tags = pd.Series(joined_tags)

    tag_counts = all_tags.value_counts().rename_axis(target_column).reset_index(name='counts')
    tag_counts['no_count'] = len(df)-tag_counts['counts']
    tag_counts['ratio'] = tag_counts['counts']/tag_counts['no_count']
    tag_counts['overall_ratio'] = tag_counts['counts']/(tag_counts['no_count'] + tag_counts['counts'])
    return tag_counts

#print(compute_tag_ratio('squash3_tags', df))
squashed_tag_counts = compute_tag_ratio('squash15_tags', df)
print_full_dataframe(squashed_tag_counts)

    squash15_tags  counts  no_count     ratio  overall_ratio
0         science    1467       846  1.734043       0.634241
1         culture    1155      1158  0.997409       0.499351
2      technology     787      1526  0.515727       0.340251
3   global issues     679      1634  0.415545       0.293558
4          design     477      1836  0.259804       0.206226
5         history     385      1928  0.199689       0.166450
6        business     349      1964  0.177699       0.150886
7   entertainment     285      2028  0.140533       0.123217
8           media     279      2034  0.137168       0.120623
9    biomechanics     220      2093  0.105112       0.095115
10         future     218      2095  0.104057       0.094250
11   biodiversity     218      2095  0.104057       0.094250
12       humanity     217      2096  0.103531       0.093818
13       politics     199      2114  0.094134       0.086035
14  communication     185      2128  0.086936       0.079983


# 3. Feature Extraction via Deep learning

## 3.1 Create one hot encoding

In [6]:
# from sklearn.preprocessing import MultiLabelBinarizer

# y = []
# for index, row in df.iterrows():
#     y.append(set(row['squash3_tags'].split(',')))
    
# mlb = MultiLabelBinarizer()
# encoded_y = mlb.fit_transform(y)

In [7]:
# print(encoded_y[0])
# print(len(encoded_y[0]))

In [8]:
joined_tags = df['squash15_tags'].str.cat(sep=',').split(',')
all_tags = pd.Series(joined_tags).str.strip().str.lower()
all_tags = list(dict.fromkeys(all_tags))
try:
    all_tags.remove('')
except:
    pass
print(all_tags)
print(len(all_tags))

['culture', 'politics', 'science', 'global issues', 'technology', 'design', 'business', 'biomechanics', 'biodiversity', 'media', 'entertainment', 'history', 'future', 'communication', 'humanity']
15


In [9]:
def create_one_hot_encode(df=df):
    complete_transcripts_tags = []
    for rows, value in df.iterrows():
        one_hot_encoding = [0] * len(all_tags)
        headline = [value['headline']]
        transcript = [value['clean_transcript_string']]
        pos_sequence = [value['pos_sequence']]
        ner_sequence = [value['ner_sequence']]
        tm = [value['tm']]
        indiv_tags = value['squash15_tags'].split(',')
        for tags in indiv_tags:
            if tags == '':
                continue
            index = all_tags.index(tags.lower().lstrip(' '))
            one_hot_encoding[index] = 1
        indiv_transcript_tags = headline + transcript + pos_sequence + ner_sequence + tm +one_hot_encoding
        complete_transcripts_tags.append(indiv_transcript_tags)
    return pd.DataFrame(complete_transcripts_tags, columns=['headline', 'transcript', 'pos_sequence', 'ner_sequence','tm'] + all_tags)

In [10]:
df = create_one_hot_encode()
df

,headline,transcript,pos_sequence,ner_sequence,tm,culture,politics,science,global issues,technology,design,business,biomechanics,biodiversity,media,entertainment,history,future,communication,humanity
0,Averting the climate crisis,thank chris truly great honor opportunity come...,VERB PROPN ADV ADJ NOUN NOUN VERB NOUN ADV ADV...,PERSON ORG ORG GPE LOC ORG PRODUCT GPE GPE PER...,"[0.04325945698517057, 0.0, 0.00142482934694180...",1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
1,Simple designs to save a life,term invention like tell tale favorite project...,NOUN NOUN SCONJ VERB PROPN ADJ NOUN VERB NOUN ...,GPE DATE CARDINAL DATE ORG PERSON LOC ORG GPE ...,"[0.013287880838036227, 0.0, 0.0, 0.00511725094...",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2,How to rebuild a broken state,public dewey long ago observe constitute discu...,ADJ PROPN ADV ADV VERB ADJ NOUN NOUN PROPN PRO...,DATE NORP ORDINAL DATE MONEY DATE DATE DATE EV...,"[0.0, 0.006699599134802422, 0.0, 0.00564851883...",1,1,1,1,0,0,1,0,0,0,0,0,0,0,0
3,The real future of space exploration,want start say houston problem enter second ge...,VERB NOUN VERB PROPN NOUN VERB ADJ NOUN NOUN N...,GPE ORDINAL ORG PERSON DATE DATE DATE TIME PER...,"[0.040282108339079505, 0.03732895646484358, 0....",0,0,1,0,0,1,1,0,0,0,0,0,0,0,0
4,Great cars are great art,want talk background idea car art actually mea...,VERB NOUN NOUN NOUN NOUN NOUN ADV ADJ NOUN NOU...,PERSON PRODUCT ORG ORG PERSON PERSON PERSON OR...,"[0.08049208168957463, 0.0, 0.0, 0.008031187136...",0,0,1,0,1,1,1,0,0,0,0,0,0,0,0
5,Sampling the ocean's DNA,break ask people comment age debate comment un...,VERB VERB NOUN VERB NOUN NOUN NOUN VERB NOUN A...,DATE DATE ORG DATE DATE PERSON ORG CARDINAL CA...,"[0.0, 0.01122282724927712, 0.0, 0.163765591818...",0,0,1,0,1,0,0,1,1,0,0,0,0,0,0
6,Simplicity sells,music sound silence simon garfunkel hello voic...,NOUN PROPN PROPN PROPN PROPN INTJ NOUN NOUN AD...,PERSON TIME TIME ORG PERSON FAC DATE DATE ORG ...,"[0.062272408748748564, 0.0, 0.0243049615007748...",0,0,1,0,1,1,0,0,0,1,1,0,0,0,0
7,A memorial at Ground Zero,kurt andersen like architect david hog limelig...,PROPN PROPN SCONJ PROPN PROPN PROPN PROPN ADV ...,PERSON PERSON ORG PERSON DATE GPE PERSON PERSO...,"[0.045631610155157765, 0.0, 0.0, 0.0, 0.004847...",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
8,To invent is to give,point time come learn morning world expert gue...,NOUN NOUN VERB VERB NOUN NOUN NOUN VERB ADJ NO...,DATE DATE TIME PERSON DATE ORG CARDINAL GPE CA...,"[0.041662618917564086, 0.0, 0.0, 0.0, 0.004215...",1,0,1,1,1,1,1,0,0,1,0,0,0,0,0
9,The killer American diet that's sweeping the p...,legitimate concern aid avian flu hear brillian...,ADJ NOUN NOUN ADJ NOUN VERB ADJ PROPN ADJ ADV ...,NORP TIME LOC LOC CARDINAL DATE DATE DATE NORP...,"[0.003366184031329983, 0.0, 0.0007976442417315...",1,0,1,1,0,0,0,0,0,0,0,0,0,0,0


In [11]:
def get_target_column(target_tag, df=df):
    return df[['headline', 'transcript','pos_sequence', 'ner_sequence','tm', target_tag]]
single_class = get_target_column('culture', df)

In [12]:
# df_x = single_class[['transcript']]
# df_y = df[['technology']]

In [13]:
df_x = single_class[['headline', 'transcript','pos_sequence', 'ner_sequence','tm']]
df_y = list(single_class['culture'])
#print(df_x[0])

## 3.2 Perform train test split

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, train_y, valid_y = train_test_split(df_x, df_y, random_state = 42)

In [15]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Conv1D, MaxPooling1D, concatenate
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras import optimizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import numpy as np

Using TensorFlow backend.


## 3.3 Use word embeddings for the main transcript

In [34]:
# Extract train and test transcripts to list 
X_train_transcripts = X_train['transcript'].tolist()
X_test_transcripts = X_test['transcript'].tolist()
# Extract headline - we will use tfidf because headlines are short 
X_train_headline = X_train['headline'].tolist()
X_test_headline = X_test['headline'].tolist()
# Extract POS tags
X_train_pos_seq= X_train['pos_sequence'].tolist()
X_test_pos_seq = X_test['pos_sequence'].tolist()
# Extract NER tags
X_train_ner_seq = X_train['ner_sequence'].tolist()
X_test_ner_seq = X_test['ner_sequence'].tolist()
# Extract topic modelling arrays
X_train_tm = X_train['tm']
X_test_tm = X_test['tm']

In [35]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train_transcripts)

X_train_transcripts = tokenizer.texts_to_sequences(X_train_transcripts)
X_test_transcripts = tokenizer.texts_to_sequences(X_test_transcripts)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 500 # since the average length is about there. Too long and the predicions are bad. we assume the intro has the most info

X_train_transcripts = pad_sequences(X_train_transcripts, padding='post', maxlen=maxlen)
X_test_transcripts = pad_sequences(X_test_transcripts, padding='post', maxlen=maxlen)

In [36]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_path = "C:/Users/JSaw/Downloads/"
glove_file = open(glove_path+'glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [37]:
print(X_train_transcripts[1])
print(X_train_transcripts.shape)
print(type(X_train_transcripts))

[4469 4298 2877  570  399  847 1837 1132 2904 1242  663 2904 1242 3117
 2877  848   67  449  583  301 2904 1242 1447 1132  261   21  570    8
   17   68 2420 1035  169   21  382 3410  399  598  101  444  101   12
 1783  715 2211  471 1378  368 2511  715  433 4299  570  153 2843  273
  438  169   76  368  438 1423 1483 1026 1285 1035  528    6   19 2453
 2683  598 1274   78  149  598 3460  598   11  813 1721 4157   11   38
   24  153 2843  678 1437 2878 1036 2033  469    5 1514 1688   17  802
  292   25 2878 1437  814 1026 1026 2454  197  387  141  814 1326  117
 1226   14  588 1233 1462  149  241 2878    8  500  102 1548  387  141
  964   30 4300   34  241 2878   51   34 3461   51 3709  448  387  141
  544  206  241 2878  625   30   14    7 1530 1186  387  141  402   69
  241 2878   21   34   47   56  136   47  241 2878   21   77  663   43
  663 1394 4552 1437 1676   10  241 2878    7  459 2810  389  888  176
 4931 4301  626  264  285  264  285   14  567  399  241  709  389  814
 2878 

In [38]:
print(train_y[0])

0


## 3.4 tfidf the headline

In [39]:
# tfidf_vect_pos = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=50)
# tfidf_vect_pos.fit(X_train_headline)

# xtrain_tfidf_headline =  tfidf_vect_pos.transform(X_train_headline)
# xtest_tfidf_headline =  tfidf_vect_pos.transform(X_test_headline)


In [40]:
# print(xtrain_tfidf_headline.shape)
# print(xtest_tfidf_headline.shape)
# print(xtrain_tfidf_headline[0])
# print(type(xtrain_tfidf_headline))

In [41]:
# tfidf_vect_pos = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
# tfidf_vect_pos.fit(df['pos_sequence'])
# tfidf_vect_ner = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
# tfidf_vect_ner.fit(df['ner_sequence'])

# xtrain_tfidf_pos =  tfidf_vect_pos.transform(X_train['pos_sequence'])
# xtest_tfidf_pos =  tfidf_vect_pos.transform(X_test['pos_sequence'])

# xtrain_tfidf_ner =  tfidf_vect_ner.transform(X_train['ner_sequence'])
# xtest_tfidf_ner =  tfidf_vect_ner.transform(X_test['ner_sequence'])

In [42]:
# Try word embeddings on the vector 
tokenizer2 = Tokenizer(num_words=100)
tokenizer2.fit_on_texts(X_train_headline)

X_train_headline = tokenizer2.texts_to_sequences(X_train_headline)
X_test_headline = tokenizer2.texts_to_sequences(X_test_headline)

vocab_size2 = len(tokenizer2.word_index) + 1

maxlen2 = 100 # since the average length is about there. Too long and the predicions are bad. we assume the intro has the most info

X_train_headline = pad_sequences(X_train_headline, padding='post', maxlen=maxlen2)
X_test_headline = pad_sequences(X_test_headline, padding='post', maxlen=maxlen2)

# 3.5 Place all tm vectors into big array

In [43]:
def compile_vectors(series,num):
    big = np.zeros((len(series),num))
    for i in range(len(series)):
        array = series.iloc[i]
        big[i] = array
        return big

In [44]:
print(type(X_train_tm))
print(type(X_test_tm))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [45]:
X_train_tm = compile_vectors(X_train_tm,15)
X_test_tm = compile_vectors(X_test_tm,15)

In [46]:
print(type(X_train_tm))
print(type(X_test_tm))
print(X_train_tm.shape)
print(X_test_tm.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1734, 15)
(579, 15)


# 3.5 POS NER

In [47]:
print(type(X_train_pos_seq))
print(type(X_test_pos_seq))

print(type(X_train_ner_seq))
print(type(X_test_ner_seq))
print(len(X_train_pos_seq[0]),len(X_train_pos_seq[1]))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
5135 3703


In [48]:
ave_pos = 0
for i in range(len(X_train_pos_seq)):
    small = len(X_train_pos_seq[i])
    ave_pos += small
for j in range(len(X_test_pos_seq)):
    small = len(X_test_pos_seq[j])
    ave_pos += small
ave_pos = int(ave_pos / (len(X_train_pos_seq)+len(X_test_pos_seq)))
print(ave_pos)

4031


In [50]:
unique_pos = list(df['pos_sequence'].str.split(' ', expand=True).stack().unique())
print(unique_pos)

['VERB', 'PROPN', 'ADV', 'ADJ', 'NOUN', 'ADP', 'SCONJ', 'INTJ', 'AUX', 'PRON', 'X', 'NUM', 'DET', 'PART', 'PUNCT', 'SYM', 'CCONJ']


In [51]:
ave_ner = 0
for i in range(len(X_train_ner_seq)):
    small = len(X_train_ner_seq[i])
    ave_ner += small
for j in range(len(X_test_ner_seq)):
    small = len(X_test_ner_seq[j])
    ave_ner += small
ave_ner = int(ave_ner / (len(X_train_ner_seq)+len(X_test_ner_seq)))
print(ave_ner)

215


In [53]:
unique_ner = list(df['ner_sequence'].str.split(' ', expand=True).stack().unique())
print(unique_ner)

['PERSON', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'DATE', 'EVENT', 'NORP', 'TIME', 'CARDINAL', 'MONEY', 'ORDINAL', 'PERCENT', 'FAC', 'QUANTITY', 'LANGUAGE', 'LAW', 'WORK_OF_ART', '']


In [52]:
# Try word embeddings on the vector 
tokenizer3 = Tokenizer(num_words=len(unique_pos))
tokenizer3.fit_on_texts(X_train_pos_seq)

X_train_pos_seq = tokenizer3.texts_to_sequences(X_train_pos_seq)
X_test_pos_seq = tokenizer3.texts_to_sequences(X_test_pos_seq)

vocab_size3 = len(tokenizer3.word_index) + 1

maxlen3 = ave_pos # since the average length is about there. Too long and the predicions are bad. we assume the intro has the most info

X_train_pos_seq = pad_sequences(X_train_pos_seq, padding='post', maxlen=maxlen3)
X_test_pos_seq = pad_sequences(X_test_pos_seq, padding='post', maxlen=maxlen3)

In [54]:
# Try word embeddings on the vector 
tokenizer4 = Tokenizer(num_words=len(unique_ner))
tokenizer4.fit_on_texts(X_train_ner_seq)

X_train_ner_seq = tokenizer4.texts_to_sequences(X_train_ner_seq)
X_test_ner_seq = tokenizer4.texts_to_sequences(X_test_ner_seq)

vocab_size4 = len(tokenizer4.word_index) + 1

maxlen4 = ave_ner # since the average length is about there. Too long and the predicions are bad. we assume the intro has the most info

X_train_ner_seq = pad_sequences(X_train_ner_seq, padding='post', maxlen=maxlen4)
X_test_ner_seq = pad_sequences(X_test_ner_seq, padding='post', maxlen=maxlen4)

# Model

In [55]:
from keras.utils import plot_model
# define two sets of inputs
inputA = Input(shape=(maxlen2,))
inputB = Input(shape=(maxlen,))
inputTM = Input(shape=(15,))
 
# the first branch operates on the first input which is the headline
embedding_layer_headline = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(inputA) 
#model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
#model.add(layers.Conv1D(128, 5, activation='relu'))
x = Conv1D(128, 5, activation='relu')(embedding_layer_headline)
# model.add(layers.GlobalMaxPooling1D())
x = GlobalMaxPooling1D()(x)
x = Dense(10, activation='relu')(x)
X = Dropout(0.2)(x)
x = Dense(4, activation="relu")(x)
# model.add(layers.Dense(10, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))

# x = Dense(50, activation="relu")(inputA)
# x = Dense(4, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)
 
# the second branch opreates on the second input
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(inputB)
y = LSTM(128)(embedding_layer)
y = Dropout(0.2)(y)
y = Dense(4, activation='relu')(y)
y = Model(inputs=inputB, outputs=y)

# third input
tm = Dense(64,activation='relu')(inputTM)
tm = Dense(4,activation='relu')(tm)
tm = Model(inputs=inputTM,outputs=tm)
 
# combine the output of the two branches
combined = concatenate([x.output, y.output, tm.output])
 
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="sigmoid")(z)
 
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input, tm.input], outputs=z)
print(model.summary())
adam = optimizers.adam(lr=0.0001)
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 100)     4267400     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)    

In [56]:
print(X_train_transcripts.shape,maxlen,type(X_train_transcripts))
print(X_train_headline.shape,maxlen2,type(X_train_headline))
print(X_train_tm.shape,15,type(X_train_tm))

(1734, 500) 500 <class 'numpy.ndarray'>
(1734, 100) 100 <class 'numpy.ndarray'>
(1734, 15) 15 <class 'numpy.ndarray'>


In [57]:
adam = optimizers.adam(lr=0.001)
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])
history = model.fit([X_train_headline, X_train_transcripts, X_train_tm], train_y, batch_size=32, epochs=4, verbose=1, validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1387 samples, validate on 347 samples
Epoch 1/4
1387/1387 [==============================] - ETA: 4:56 - loss: 0.7227 - acc: 0.437 - ETA: 2:39 - loss: 0.7083 - acc: 0.484 - ETA: 2:01 - loss: 0.6994 - acc: 0.510 - ETA: 1:36 - loss: 0.7113 - acc: 0.429 - ETA: 1:21 - loss: 0.7087 - acc: 0.425 - ETA: 1:15 - loss: 0.7062 - acc: 0.432 - ETA: 1:07 - loss: 0.7040 - acc: 0.437 - ETA: 1:00 - loss: 0.7028 - acc: 0.445 - ETA: 56s - loss: 0.7016 - acc: 0.465 - ETA: 51s - loss: 0.7006 - acc: 0.48 - ETA: 47s - loss: 0.7000 - acc: 0.49 - ETA: 46s - loss: 0.6995 - acc: 0.49 - ETA: 43s - loss: 0.6990 - acc: 0.49 - ETA: 40s - loss: 0.6986 - acc: 0.49 - ETA: 39s - loss: 0.6983 - acc: 0.49 - ETA: 37s - loss: 0.6978 - acc: 0.50 - ETA: 35s - loss: 0.6976 - acc: 0.49 - ETA: 33s - loss: 0.6972 - acc: 0.49 - ETA: 31s - loss: 0.6969 - acc: 0.49 - ETA: 29s - loss: 0.6969 - acc: 0.

In [ ]:
# from keras.utils import plot_model
# # define two sets of inputs
# inputA = Input(shape=(50,))
# inputB = Input(shape=(maxlen,))
 
# # the first branch operates on the first input
# x = Dense(50, activation="relu")(inputA)
# x = Dense(4, activation="relu")(x)
# x = Model(inputs=inputA, outputs=x)
 
# # the second branch opreates on the second input
# embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(inputB)
# y = LSTM(128)(embedding_layer)
# y = Dense(4, activation='sigmoid')(y)
# y = Model(inputs=inputB, outputs=y)
 
# # combine the output of the two branches
# combined = concatenate([x.output, y.output])
 
# # apply a FC layer and then a regression prediction on the
# # combined outputs
# z = Dense(2, activation="relu")(combined)
# z = Dense(1, activation="linear")(z)
 
# # our model will accept the inputs of the two branches and
# # then output a single value
# model = Model(inputs=[x.input, y.input], outputs=z)
# print(model.summary())
# adam = optimizers.adam(lr=0.0001)
# #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
# model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])

In [58]:
plot_model(model, to_file='model_plot_cnn_tm.png', show_shapes=True, show_layer_names=True)

In [ ]:
# adam = optimizers.adam(lr=0.0001)
# #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
# model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])
# history = model.fit([xtrain_tfidf_headline, X_train_transcripts], train_y, batch_size=32, epochs=4, verbose=1, validation_split=0.2)

In [ ]:
# from keras.utils import plot_model
# plot_model(model)

In [ ]:
# deep_inputs = Input(shape=(maxlen,))
# embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
# LSTM_Layer_1 = LSTM(128)(embedding_layer)
# dense_layer_1 = Dense(1, activation='sigmoid')(LSTM_Layer_1)
# model = Model(inputs=deep_inputs, outputs=dense_layer_1)

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
# history = model.fit(X_train, train_y, batch_size=128, epochs=6, verbose=1, validation_split=0.2)

In [ ]:
# model_glove = Sequential()
# model_glove.add(Embedding(vocab_size, 100, input_length=3000, weights=[embedding_matrix], trainable=False))(Input(shape=(maxlen,)))
# model_glove.add(Dropout(0.2))
# model_glove.add(Conv1D(64, 5, activation='relu'))
# model_glove.add(MaxPooling1D(pool_size=4))
# model_glove.add(LSTM(100))
# model_glove.add(Dense(1, activation='sigmoid'))
# model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# ## Fit train data
# model_glove.fit(X_train, np.array(train_y), validation_split=0.2, epochs = 3)

In [59]:
predictions = model.predict([X_test_headline, X_test_transcripts, X_test_tm])

In [ ]:
print(predictions)

In [60]:
def get_tag(threshold, predictions=predictions):
    return [[1 if j > threshold else 0 for j in i.tolist()] for i in predictions]

def get_tag_flat(threshold, predictions=predictions):
    return [1 if j > threshold else 0 for i in predictions for j in i]
predictions_flushed = get_tag(0.4)


In [61]:
def compute_tp_tn_fp_fn(y_test, y_pred, classes):
    '''
    Return:
    pre_score = {
        'tag_1': {
            'index': ,
            'tp': ,
            'tn': ,
            'fp': ,
            'fn': 
        }
    }
    '''
    # Create dictionary of tags 
    pre_score = {}
    for index_tag, tag in enumerate(classes):
        pre_score[tag] = {
            'index':index_tag,
            'tp': 0,
            'tn': 0,
            'fp': 0,
            'fn': 0
        }
    for transcript_index, transcript_value in enumerate(y_test):
        if transcript_value == y_pred[transcript_index][0] and transcript_value == 1:
            pre_score[classes[0]]['tp'] += 1
        elif transcript_value == y_pred[transcript_index][0] and transcript_value == 0:
            pre_score[classes[0]]['tn'] += 1
        elif transcript_value != y_pred[transcript_index][0] and transcript_value == 1:
            pre_score[classes[0]]['fn'] += 1
        elif transcript_value != y_pred[transcript_index][0] and transcript_value == 0:
            pre_score[classes[0]]['fp'] += 1
    return pre_score
scores_preprocess = compute_tp_tn_fp_fn(valid_y, predictions_flushed, ['culture'])

In [62]:
def compute_precision_recall_f1(preprocessed_scores):
    for key, value in preprocessed_scores.items():
        try:
            precision = value['tp']/(value['tp']+value['fp'])
        except:
            print('precision issue: {}'.format(key))
            precision = 0.0
        try:
            recall = value['tp']/(value['tp']+value['fn'])
        except:
            print('recall issue: {}'.format(key))
            recall = 0.0
        try:
            f1 = (2 * precision * recall)/(precision + recall)
        except:
            print('f1 issue: {}'.format(key))
            f1=0.0
        preprocessed_scores[key]['precision'] = round(precision,2)
        preprocessed_scores[key]['recall'] = round(recall,2)
        preprocessed_scores[key]['f1'] = round(f1,2)
    return preprocessed_scores
final_scores = compute_precision_recall_f1(scores_preprocess)
print(final_scores)

{'culture': {'index': 0, 'tp': 291, 'tn': 16, 'fp': 263, 'fn': 9, 'precision': 0.53, 'recall': 0.97, 'f1': 0.68}}


In [63]:
def print_full_dataframe(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [64]:
def format_scores_df(tag_classes, final_scores=final_scores):
    precision = []
    recall = []
    f1 = []
    accuracy = []
    for index, value in enumerate(tag_classes):
        precision.append(final_scores[value]['precision'])
        recall.append(final_scores[value]['recall'])
        f1.append(final_scores[value]['f1'])
        accuracy.append((final_scores[value]['tp'] + final_scores[value]['tn'])/(final_scores[value]['tp'] + final_scores[value]['tn'] + final_scores[value]['fp'] + final_scores[value]['fn']))
    df_result = pd.DataFrame(list(zip(tag_classes, precision, recall, f1, accuracy)), 
               columns =['class', 'precision', 'recall', 'f1', 'accuracy']) 
    return df_result
df_results = format_scores_df(['culture'], final_scores)
print_full_dataframe(df_results)

     class  precision  recall    f1  accuracy
0  culture       0.53    0.97  0.68  0.530225


In [65]:
highest_f1 = 0
f1_i = []
highest_accuracy_f1 = 0
accuracy_f1_i = []
highest_accuracy = 0
accuracy_i = []
for i in range(25, 45):
    i = i/100
#     print(i)
    predictions_flushed = get_tag(i)
    scores_preprocess = compute_tp_tn_fp_fn(valid_y, predictions_flushed, ['culture'])
    final_scores = compute_precision_recall_f1(scores_preprocess)
#     print(final_scores)
    df_results = format_scores_df(['culture'], final_scores)
    print(df_results)
    f1 = final_scores['culture']['f1']
    accuracy = df_results.accuracy[0]
    
    if f1 > highest_f1:
        highest_f1 = f1
        f1_i = [i]
        if accuracy > highest_accuracy_f1:
            highest_accuracy_f1 = accuracy
            accuracy_f1_i = [i]
        elif accuracy == highest_accuracy_f1:
            accuracy_f1_i.append(i)
    elif f1 == highest_f1:
        f1_i.append(i)
        if accuracy > highest_accuracy_f1:
            highest_accuracy_f1 = accuracy
            accuracy_f1_i = [i]
        elif accuracy == highest_accuracy_f1:
            accuracy_f1_i.append(i)
    
    if accuracy > highest_accuracy:
        highest_accuracy = accuracy
        accuracy_i = [i]
    elif accuracy == highest_accuracy:
        accuracy_i.append(i)

#     print('\n')

print(highest_f1,f1_i)
print(highest_accuracy_f1,accuracy_f1_i)
print(highest_accuracy,accuracy_i)

     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.516408
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.516408
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.516408
     class  precision  recall    f1  accuracy
0  culture       0.52    0.99  0.68   0.51468
     class  precision  recall    f1  accuracy
0  culture       0.52    0.99  0.68   0.51468
     class  precision  recall    f1  accuracy
0  culture       0.52    0.99  0.6

In [66]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
def evaluate_on_training_set(y_test, y_pred):
  # Calculate AUC
  print("AUC is: ", roc_auc_score(y_test, y_pred))
  # print out recall and precision
  print(classification_report(y_test, y_pred))
  # print out confusion matrix
  print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
  # # calculate points for ROC curve
  fpr, tpr, thresholds = roc_curve(y_test, y_pred)
  # Plot ROC curve
  plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc_score(y_test, y_pred)) 
  plt.plot([0, 1], [0, 1], 'k--') # random predictions curve
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.0])
  plt.xlabel('False Positive Rate or (1 - Specifity)')
  plt.ylabel('True Positive Rate or (Sensitivity)')
  plt.title('Receiver Operating Characteristic')

In [ ]:
evaluate_on_training_set(valid_y, get_tag_flat(0.35))
#print(valid_y)